In [ ]:
import pandas as pd
from collections import Counter
import re
import spacy
import pycountry
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_data = pd.read_csv("/media/moraa/New Volume/Ontita/10Academy/Cohort B/Projects/week0/Datasets/data.csv/rating.csv")
df_raw = pd.read_csv("/media/moraa/New Volume/Ontita/10Academy/Cohort B/Projects/week0/Datasets/raw_data/data.csv")
df_traffic = pd.read_csv("/media/moraa/New Volume/Ontita/10Academy/Cohort B/Projects/week0/Datasets/traffic_data/traffic.csv")
df_domains = pd.read_csv("/media/moraa/New Volume/Ontita/10Academy/Cohort B/Projects/week0/Datasets/domains_location.csv")

In [ ]:
df_data.head()

In [ ]:
df_raw.head()

In [ ]:
df_traffic.head()

In [ ]:
df_domains.head()

# EDA

Top and Bottom Websites that have the largest count of news articles

In [ ]:
df_data['source_name'].head(10)

In [ ]:
# Set pandas display options to show all rows
pd.set_option('display.max_rows', None)

# Count the number of news articles for each website
website_article_counts = df_data['source_name'].value_counts()

# Convert the series to a DataFrame for easier manipulation
website_article_counts_df = pd.DataFrame(website_article_counts)
website_article_counts_df.reset_index(inplace=True)
website_article_counts_df.columns = ['Website', 'Article Count']

# Sort the websites based on their article counts in descending order to find the top 10
top_10_websites = website_article_counts_df.nlargest(10, 'Article Count')

# Sort the websites based on their article counts in ascending order to find the bottom 10
bottom_10_websites = website_article_counts_df.nsmallest(10, 'Article Count')

print("Top 10 websites with the largest count of news articles:")
print(top_10_websites)


In [ ]:
print("\nBottom 10 websites with the smallest count of news articles:")
print(bottom_10_websites)

Top and Bottom 10 Websites with the highest numbers of visitors traffic

In [ ]:
# Sorting the DataFrame by GlobalRank in ascending order to get the top 10
top_10_websites = df_traffic.sort_values(by='GlobalRank').head(10)

# Sorting the DataFrame by GlobalRank in descending order to get the bottom 10
bottom_10_websites = df_traffic.sort_values(by='GlobalRank', ascending=False).head(10)

print("Top 10 Websites with the Highest Visitor Traffic:")
print(top_10_websites[['Domain', 'GlobalRank']])

print("\nBottom 10 Websites with the Lowest Visitor Traffic:")
print(bottom_10_websites[['Domain', 'GlobalRank']])

Top and Bottom 10 Countries with the highest number of news media organisations (represented by domains in the data)

In [ ]:
# Grouping the DataFrame by 'location' and counting the number of unique domains for each country
country_counts = df_domains.groupby('location')['SourceCommonName'].nunique().reset_index()

# Sorting the DataFrame by the counts of unique domains in descending order to get the top 10
top_10_countries = country_counts.sort_values(by='SourceCommonName', ascending=False).head(10)

# Sorting the DataFrame by the counts of unique domains in ascending order to get the bottom 10
bottom_10_countries = country_counts.sort_values(by='SourceCommonName').head(10)

print("Top 10 Countries with the Highest Number of News Media Organizations:")
print(top_10_countries)

print("\nBottom 10 Countries with the Lowest Number of News Media Organizations:")
print(bottom_10_countries)

Top and Bottom 10 Countries that have many articles written about them - the content of the news is about that country

In [ ]:

df_data['content'].head()

In [ ]:
# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Function to extract country mentions from text using spaCy
def extract_countries(text):
    doc = nlp(text)
    countries = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
    return countries

# Apply the function to extract country mentions from the content column
df_data['country_mentions'] = df_data['content'].apply(extract_countries)

# Flatten the list of country mentions
country_mentions_flat = [country for sublist in df_data['country_mentions'] for country in sublist]

# Count the occurrences of each country mention
country_counts = pd.Series(country_mentions_flat).value_counts()

# Display the top and bottom 10 countries mentioned in the articles
top_10_countries = country_counts.head(10)
bottom_10_countries = country_counts.tail(10)

print("Top 10 countries mentioned in the articles:")
print(top_10_countries)

print("\nBottom 10 countries mentioned in the articles:")
print(bottom_10_countries)

Top and Bottom Websites that reported (the news content) about Africa, US, China, EU, Russia, Ukraine, Middle East? Note that you will need to group countries together to form the African, EU, and Middle East continents/regions.

In [ ]:
df_domains['Country'].head()

In [ ]:
# List of countries, regions, and continents
countries_list = [
    'us', 'usa', 'united states', 'united states of america', 'america', 'united states of',
    'china', 'prc', 'people\'s republic of china',
    'russia', 'russian federation', 'soviet union', 'ussr',
    'ukraine', 'ukr', 'ukrainian',
    'middle east', 'middle eastern', 'mideast',
    'africa', 'african',
    'north america', 'north american', 'na',  # For USA
    'asia', 'asian', 'cn'  # For China
]

# Function to extract countries/regions from text
def extract_countries(text):
    doc = nlp(text)
    countries = [ent.text.lower() for ent in doc.ents if ent.label_ == "GPE" and ent.text.lower() in countries_list]
    return countries

# Apply the function to extract countries from content column
df_data['mentioned_countries'] = df_data['content'].apply(extract_countries)

# Count mentions of each country/region
df_data['mentioned_countries_count'] = df_data['mentioned_countries'].apply(len)

# Group by source_name and sum the counts of mentions
grouped_by_source = df_data.groupby('source_name')['mentioned_countries_count'].sum()

# Sort the groups by counts of mentions
sorted_sources = grouped_by_source.sort_values(ascending=False)

# Extract top and bottom 10 sources
top_10_sources = sorted_sources.head(10)
bottom_10_sources = sorted_sources.tail(10)

print("Top 10 sources reporting news about the regions:")
print(top_10_sources)
print("\nBottom 10 sources reporting news about the regions:")
print(bottom_10_sources)

Top and Bottom 10 Websites with the highest count of positive, neutral, and negative sentiment?

In [ ]:
df_data['url'].head()

In [ ]:
# Function to extract domain from URL
def extract_domain(url):
    try:
        parsed_uri = urlparse(url)
        if parsed_uri.scheme == '' or parsed_uri.netloc == '':
            return None
        domain = '{uri.netloc}'.format(uri=parsed_uri)
        return domain
    except:
        return None

# Apply the function to extract domain from the 'url' column
df_data['domain'] = df_data['url'].apply(extract_domain)

# Group the data by website domain and sentiment, then count the occurrences
sentiment_counts = df_data.groupby(['domain', 'title_sentiment']).size().unstack(fill_value=0)

# Calculate total count of sentiments for each website
sentiment_counts['Total'] = sentiment_counts.sum(axis=1)

# Sort the websites based on total count of each sentiment
top_bottom_website_sentiments = sentiment_counts.sort_values(by='Total', ascending=False)

# Extract top and bottom 10 websites for each sentiment
top_positive = top_bottom_website_sentiments.nlargest(10, 'Positive')['Positive']
top_neutral = top_bottom_website_sentiments.nlargest(10, 'Neutral')['Neutral']
top_negative = top_bottom_website_sentiments.nlargest(10, 'Negative')['Negative']

bottom_positive = top_bottom_website_sentiments.nsmallest(10, 'Positive')['Positive']
bottom_neutral = top_bottom_website_sentiments.nsmallest(10, 'Neutral')['Neutral']
bottom_negative = top_bottom_website_sentiments.nsmallest(10, 'Negative')['Negative']

print("Top 10 Websites with the highest count of positive sentiment:")
print(top_positive)
print("\nTop 10 Websites with the highest count of neutral sentiment:")
print(top_neutral)
print("\nTop 10 Websites with the highest count of negative sentiment:")
print(top_negative)

print("\nBottom 10 Websites with the lowest count of positive sentiment:")
print(bottom_positive)
print("\nBottom 10 Websites with the lowest count of neutral sentiment:")
print(bottom_neutral)
print("\nBottom 10 Websites with the lowest count of negative sentiment:")
print(bottom_negative)

Comparing the impact of using mean/average and median

In [ ]:
# Group the data by website domain and sentiment, then calculate mean sentiment counts
mean_sentiment_counts = df_data.groupby('domain')['title_sentiment'].value_counts(normalize=True).unstack(fill_value=0)

# Calculate mean sentiment counts for each website
mean_sentiment_counts['Mean_Total'] = mean_sentiment_counts.sum(axis=1)

# Sort the websites based on mean total count of each sentiment
mean_top_bottom_websites = mean_sentiment_counts.sort_values(by='Mean_Total', ascending=False)

# Repeat the same process for median sentiment counts
median_sentiment_counts = df_data.groupby('domain')['title_sentiment'].value_counts().unstack(fill_value=0)
median_sentiment_counts['Median_Total'] = median_sentiment_counts.sum(axis=1)
median_top_bottom_websites = median_sentiment_counts.sort_values(by='Median_Total', ascending=False)

# Compare the top and bottom websites based on mean vs median sentiment counts for each sentiment
for sentiment in ['Positive', 'Neutral', 'Negative']:
    print(f"Top 10 Websites with the highest count of {sentiment.lower()} sentiment (using Mean):")
    print(mean_top_bottom_websites[sentiment].nlargest(10))
    print(f"\nTop 10 Websites with the highest count of {sentiment.lower()} sentiment (using Median):")
    print(median_top_bottom_websites[sentiment].nlargest(10))
    print("\n")

    print(f"Bottom 10 Websites with the lowest count of {sentiment.lower()} sentiment (using Mean):")
    print(mean_top_bottom_websites[sentiment].nsmallest(10))
    print(f"\nBottom 10 Websites with the lowest count of {sentiment.lower()} sentiment (using Median):")
    print(median_top_bottom_websites[sentiment].nsmallest(10))
    print("\n")


Check the distribution of Sentiments for a particular domain

In [ ]:
df_data['domain'].head()

In [ ]:
# Group by domain and calculate the number of articles for each domain
domain_article_counts = df_data['domain'].value_counts()

# Select the top 10 domains by number of articles
top_10_domains = domain_article_counts.nlargest(10).index

# Filter df_data to include only articles from the top 10 domains
df_top_10_domains = df_data[df_data['domain'].isin(top_10_domains)]

# Calculate sentiment distribution for each domain
domain_sentiment_distribution = df_top_10_domains.groupby(['domain', 'title_sentiment']).size().unstack(fill_value=0)

# Calculate global news sentiment distribution
global_sentiment_distribution = df_data['title_sentiment'].value_counts(normalize=True) * 100

# Print the sentiment distributions for each domain and the global news sentiment distribution
print("Sentiment Distribution for Top 10 Domains:")
print(domain_sentiment_distribution)

print("\nGlobal News Sentiment Distribution:")
print(global_sentiment_distribution)

How similar are the raw message lengths across sites

In [ ]:
# Extract relevant data and create a copy to avoid the warning
relevant_data = df_data[['source_name', 'content']].copy()

# Calculate raw message lengths using .loc to set values explicitly
relevant_data.loc[:, 'message_length'] = relevant_data['content'].apply(lambda x: len(str(x)))

# Group by Site and Calculate Descriptive Statistics
stats_by_site = relevant_data.groupby('source_name')['message_length'].describe()

# 4. Visualize the Distribution
plt.figure(figsize=(10, 6))
boxplot = relevant_data.boxplot(column='message_length', by='source_name', patch_artist=True)
plt.title('Distribution of Raw Message Lengths Across Sites')
plt.suptitle('')
plt.xlabel('Site')
plt.ylabel('Message Length')
plt.xticks(rotation=45)
plt.show()

# Alternatively, you can also plot histograms for better understanding of the distribution
plt.figure(figsize=(10, 6))
for site, data in relevant_data.groupby('source_name'):
    plt.hist(data['message_length'], bins=50, alpha=0.5, label=site, density=True, range=(0, 2000))  
plt.title('Distribution of Raw Message Lengths Across Sites')
plt.xlabel('Message Length')
plt.ylabel('Density')
plt.legend()
plt.show()

How similar are the number of words in the title across sites

In [ ]:
# Extract relevant data and create a copy to avoid the warning
relevant_data = df_data[['source_name', 'title']].copy()

# Calculate the number of words in the title
relevant_data['title_word_count'] = relevant_data['title'].apply(lambda x: len(str(x).split()))

# Plot histograms with adjusted scale (increased number of bins and adjusted range)
plt.figure(figsize=(10, 6))
for site, data in relevant_data.groupby('source_name'):
    plt.hist(data['title_word_count'], bins=20, alpha=0.5, label=site, density=True)  
plt.title('Distribution of Title Word Counts Across Sites')
plt.xlabel('Number of Words in Title')
plt.ylabel('Density')
plt.legend()
plt.show()

The impact of frequent news reporting and sentiment to the website’s global ranking

In [ ]:
df_traffic.head()

In [ ]:
# Calculate total number of reports by each website
report_counts = df_data['source_name'].value_counts().reset_index()
report_counts.columns = ['source_name', 'report_count']

# Convert sentiment strings to numerical values
sentiment_mapping = {'Positive': 1, 'Negative': -1, 'Neutral': 0}
df_data['title_sentiment_numeric'] = df_data['title_sentiment'].map(sentiment_mapping)

# Aggregate average sentiment for each website
avg_sentiment = df_data.groupby('source_name')['title_sentiment_numeric'].mean().reset_index()
avg_sentiment.columns = ['source_name', 'avg_sentiment']

# Aggregate global ranking for each website
global_ranking = df_traffic.groupby('Domain')['GlobalRank'].mean().reset_index()
global_ranking.columns = ['source_name', 'GlobalRank']

# Merge aggregated data
merged_data = pd.merge(report_counts, avg_sentiment, on='source_name', how='outer')
merged_data = pd.merge(merged_data, global_ranking, on='source_name', how='outer')

plt.figure(figsize=(10, 6))
plt.scatter(merged_data['report_count'], merged_data['GlobalRank'], c=merged_data['avg_sentiment'], cmap='coolwarm')
plt.colorbar(label='Average Sentiment')
plt.xlabel('Total Number of Reports')
plt.ylabel('Global Ranking')
plt.title('Impact of News Reporting Frequency and Sentiment on Website Global Ranking')
plt.grid(True)
plt.show()

# Data Preparation

## Data.csv

In [ ]:
df_data.shape

In [ ]:
df_data.isnull().sum()

### Handling Missing Values

In [ ]:
# Calculating the percentage of missing values
missing_percentage = (df_data.isnull().sum() / len(df_data)) * 100
print(missing_percentage)

In [ ]:
# Drop the source_id column as it contain 69% missing values
df_data = df_data.drop(columns=['source_id'])

# Replace missing values in text columns with 'Unknown'
df_data['author'].fillna('Unknown', inplace=True)
df_data['description'] = df_data['description'].fillna('Not Available')
df_data['url_to_image'] = df_data['url_to_image'].fillna('Not Available')
df_data['category'] = df_data['category'].fillna('Unknown')

## Domain_location.csv

In [ ]:
df_domains.isnull().sum()

### Handling Missing Value

In [ ]:
# Mode imputation for the 'Country' column
mode_country = df_domains['Country'].mode()[0]
df_domains['Country'].fillna(mode_country, inplace=True)

## Traffic_data.csv

In [ ]:
df_traffic.isnull().sum()

In [ ]:
df_domains['Country'].head()

# Text Preprocessing

Keyword extraction/modelling using TF-IDF

In [ ]:
nltk.download('wordnet')

# Function to preprocess text
def preprocess_text(text):
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove emojis
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Remove emoticons
    text = re.sub(r':\)|;\)|:-\)|:-\(|:-D|:-\(|:-\)|:D|:P|:S|:\||:O|:\(|:\-D|:\-S', '', text)
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [ ]:
# Function to perform TF-IDF vectorization and keyword extraction
def extract_keywords(df_data):
    # Apply text preprocessing to 'content' column
    df_data['clean_content'] = df_data['content'].apply(preprocess_text)
    
    # Perform TF-IDF vectorization
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_data['clean_content'])

    # Get feature names (words)
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Create a DataFrame from TF-IDF matrix
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Identify top keywords
    top_keywords = tfidf_df.sum().sort_values(ascending=False).head(10)
    return top_keywords


# Sample data preprocessing and keyword extraction
top_keywords = extract_keywords(df_data)

# Display top keywords
print("Top Keywords:", top_keywords)

Similarity Search

In [ ]:
# Convert values in the 'title' column to strings only if they are convertible
df_data['title'] = df_data['title'].apply(lambda x: str(x) if isinstance(x, str) else x)


In [ ]:
print(df_data['title'].apply(type).value_counts())

In [ ]:
# Check the data type of the 'title' column
print(df_data['title'].dtype)

In [ ]:
# Function to calculate cosine similarity between headline/title and news body for a subset of articles
def calculate_similarity_subset(df_data, sample_size=1000):
    # Sample subset of articles
    df_sample = df_data.sample(n=sample_size, random_state=42)
    
    # Apply text preprocessing to 'title' and 'content' columns
    df_sample['clean_title'] = df_sample['title'].apply(preprocess_text)
    df_sample['clean_content'] = df_sample['content'].apply(preprocess_text)
    
    # Perform TF-IDF vectorization for title
    tfidf_vectorizer_title = TfidfVectorizer(max_features=1000)
    tfidf_matrix_title = tfidf_vectorizer_title.fit_transform(df_sample['clean_title'])
    
    # Perform TF-IDF vectorization for content
    tfidf_vectorizer_content = TfidfVectorizer(max_features=1000)
    tfidf_matrix_content = tfidf_vectorizer_content.fit_transform(df_sample['clean_content'])
    
    # Calculate cosine similarity between title and content vectors
    similarity_scores = cosine_similarity(tfidf_matrix_title, tfidf_matrix_content)
    
    return similarity_scores

# Calculate similarity scores for a subset of articles
similarity_scores_subset = calculate_similarity_subset(df_data)

# Aggregate similarity scores across all articles
overall_similarity_subset = similarity_scores_subset.mean()

print("Overall similarity between keywords in headline/title and news body across sites (subset):", overall_similarity_subset)

Topic Modelling

In [ ]:
# Function to perform topic modeling using LDA
def perform_topic_modeling(df_data, num_topics=5):
    # Apply text preprocessing to 'content' column
    df_data['clean_content'] = df_data['content'].apply(preprocess_text)
    
    # Create document-term matrix using CountVectorizer
    vectorizer = CountVectorizer(max_features=1000, stop_words='english')
    dtm = vectorizer.fit_transform(df_data['clean_content'])
    
    # Fit LDA model
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(dtm)
    
    return lda_model, vectorizer

# Perform topic modeling
num_topics = 5  # You can adjust the number of topics as needed
lda_model, vectorizer = perform_topic_modeling(df_data, num_topics)

# Function to display top words for each topic
def display_topics(model, vectorizer, num_top_words=10):
    feature_names = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))
        print()

# Display top words for each topic
display_topics(lda_model, vectorizer)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Step 1: Text Preprocessing (assuming you have already preprocessed the text data)
preprocessed_text = df_data['content'].apply(preprocess_text)

# Step 2: Create Document-Term Matrix
vectorizer = CountVectorizer(max_features=1000, stop_words='english')  # Adjust max_features as needed
dtm = vectorizer.fit_transform(preprocessed_text)

# Step 3: Apply LDA Model
num_topics = 10  # Adjust the number of topics as needed
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model.fit(dtm)

# Step 4: Interpret Topics
# Get the top words for each topic
feature_names = vectorizer.get_feature_names_out()
num_top_words = 10  # Number of top words to display for each topic
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx+1}:")
    top_words_idx = topic.argsort()[:-num_top_words-1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(top_words)


Categorizing the title/content

In [ ]:
def categorize_topic(title, content):
    categories = {
        'Breaking News': ['breaking news', 'urgent news', 'emergency'],
        'Politics': ['politics', 'government', 'election', 'president', 'political parties'],
        'World News': ['world news', 'international news', 'global events'],
        'Business/Finance': ['business', 'finance', 'economy', 'market', 'stock', 'investment', 'company', 'entrepreneur'],
        'Technology': ['technology', 'innovation', 'computer', 'software'],
        'Science': ['science', 'research', 'discovery', 'experiment', 'physics', 'biology', 'chemistry', 'astronomy'],
        'Health': ['health', 'healthcare', 'wellness', 'medicine', 'nutrition', 'fitness', 'disease', 'pandemic'],
        'Entertainment': ['movie', 'film', 'actor', 'actress', 'music', 'celebrity', 'entertainment', 'Hollywood'],
        'Sports': ['football', 'soccer', 'basketball', 'tennis', 'cricket'],
        'Environment': ['environment', 'climate change', 'sustainability', 'pollution', 'conservation', 'ecosystem', 'wildlife'],
        'Crime': ['crime', 'law enforcement', 'legal issues', 'criminal activity'],
        'Education': ['education', 'school', 'university', 'student', 'teacher', 'learning', 'curriculum', 'academic'],
        'Weather': ['weather', 'forecast', 'temperature', 'meteorology', 'climate', 'storm', 'rain', 'snow'],
        'Other': [] 
    }


    # Initialize category to None
    category = None
    
    # Check for matches in title
    for cat, keywords in categories.items():
        for keyword in keywords:
            if keyword in title.lower():
                category = cat
                break
        if category:
            break
            
    # If no match found in title, check content
    if not category:
        for cat, keywords in categories.items():
            for keyword in keywords:
                if keyword in content.lower():
                    category = cat
                    break
            if category:
                break
    
    # If category is still None, assign it as "Others"
    if not category:
        category = "Others"
                
    return category

# Apply categorization function to create new column 'topic_category'
df_data['topic_category'] = df_data.apply(lambda row: categorize_topic(row['title'], row['content']), axis=1)

# Display the DataFrame with the new 'topic_category' column
print(df_data.head())


In [ ]:
df_data.head()

Analyzing topic and trends

Most diverse website

In [ ]:
df_data['clean_content'] = df_data['content'].apply(preprocess_text)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_data['clean_content'], df_data['topic_category'], test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed

# Fit and transform TF-IDF vectorizer on training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train a classifier
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_tfidf, y_train)

# Evaluate the classifier
y_pred = classifier.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
# Step 2: Calculate Diversity Score
# Calculate diversity score for each website
df_data['topic_category'] = classifier.predict(tfidf_vectorizer.transform(df_data['clean_content']))

# Group by website and count unique categories
website_diversity = df_data.groupby('source_name')['topic_category'].nunique()

# Step 3: Identify Websites with High Diversity
most_diverse_websites = website_diversity.sort_values(ascending=False).head(10)
print("Top 10 most diverse websites:")
print(most_diverse_websites)

In [ ]:
# Step 4: Analyze Trends
# Convert 'published_at' column to datetime
df_data['published_at'] = pd.to_datetime(df_data['published_at'], errors='coerce')

# Extract year and month from 'published_at' column
df_data['year_month'] = df_data['published_at'].dt.to_period('M')

# Group by year_month and source_name, count unique categories
diversity_trends = df_data.groupby(['year_month', 'source_name'])['topic_category'].nunique().unstack()

# Step 5: Visualize Results
plt.figure(figsize=(10, 6))
sns.barplot(x=most_diverse_websites.index, y=most_diverse_websites.values)
plt.xticks(rotation=45)
plt.xlabel('Website')
plt.ylabel('Number of Unique Categories')
plt.title('Top 10 Most Diverse Websites')
plt.show()

Analyze topic trends

In [ ]:
# Extract date from 'published_at' column
df_data['date'] = df_data['published_at'].dt.date

# Group by date and topic_category, then count occurrences
topic_counts = df_data.groupby(['date', 'topic_category']).size().reset_index(name='count')

# Pivot the table to get counts of each topic on each date
topic_counts_pivot = topic_counts.pivot(index='date', columns='topic_category', values='count').fillna(0)

# Define a color palette with at least 14 distinct colors
colors = sns.color_palette('tab20', n_colors=14)

# Plotting with a custom color palette
plt.figure(figsize=(12, 8))
sns.scatterplot(data=topic_counts_pivot, markers='o', palette=colors, alpha=0.8)
plt.xlabel('Date')
plt.ylabel('Topics')
plt.title('Topic Trends Over Time')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

*Observed Trends*

It seems that alot of people read news for five continuous days then skip two days probably the Weekend then peaks up again from Monday. The most read news article is about business and politics.

In [ ]:
category_freq = df_data['category'].value_counts()
topic_category_freq = df_data['topic_category'].value_counts()

plt.figure(figsize=(10, 6))
category_freq.plot(kind='bar', color='skyblue', alpha=0.7, label='Category')
topic_category_freq.plot(kind='bar', color='orange', alpha=0.7, label='Topic Category')
plt.xlabel('Category')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Categories')
plt.legend()
plt.xticks(rotation=45)
plt.show()

Modelling the events

In [ ]:
# Preprocess text and vectorize using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_data['content'])

# Apply K-means clustering
num_clusters = 13  # Choose the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
pipeline = make_pipeline(Normalizer(), kmeans)
pipeline.fit(tfidf_matrix)
clusters = kmeans.labels_ 

# Get cluster labels
cluster_labels = pipeline.predict(tfidf_matrix)

# Add cluster labels to DataFrame
df_data['cluster'] = cluster_labels


In [ ]:
# Define sparse TF-IDF matrix
sparse_tfidf_matrix = csr_matrix(tfidf_matrix)

# Dimensionality Reduction for Visualization using TruncatedSVD
svd = TruncatedSVD(n_components=2)
reduced_features = svd.fit_transform(sparse_tfidf_matrix)

# Visualize the Clusters
df_visualize = pd.DataFrame(reduced_features, columns=['PC1', 'PC2'])
df_visualize['cluster'] = clusters

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_visualize, x='PC1', y='PC2', hue='cluster', palette='bright', legend='full')
plt.title('2D Scatter Plot of Clusters')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

13 events are covered in the data

News site that reported events the earliest

In [ ]:

# Convert 'published_at' to datetime format
df_data['published_at'] = pd.to_datetime(df_data['published_at'])

# Group by source name and find the earliest publication date for each source
earliest_publication_dates = df_data.groupby('source_name')['published_at'].min()

# Sort the news sites based on their earliest publication date
earliest_publication_dates = earliest_publication_dates.sort_values()

# Display the news sites and their earliest publication dates
print(earliest_publication_dates)


Which events have the highest reporting

In [ ]:

# Count the occurrences of each unique event
event_counts = df_data['title'].value_counts()

# Sort the events based on their occurrence counts
sorted_events = event_counts.sort_values(ascending=False)

# Display the events with the highest reporting
print("Events with the highest reporting:")
print(sorted_events.head(10))  # Adjust the number based on how many top events you want to see


Correlation between news sites reporting events

In [ ]:
# Filter data to include only articles related to events
event_articles = df_data[df_data['content'].notna()]

# Aggregate data by news source to get a summary of each source's reporting activities
reporting_summary = event_articles.groupby('source_name').size().reset_index(name='event_count')

from sklearn.preprocessing import StandardScaler

#let's use the frequency of reporting as the feature
# reporting_summary = df_data.groupby('source_name').size().reset_index(name='event_count')

# Scale the feature(s) to ensure they have the same influence on clustering
scaler = StandardScaler()
X = scaler.fit_transform(reporting_summary[['event_count']])

# Choose the number of clusters (k)
k = 3  # You may need to adjust this based on your data and desired number of clusters
kmeans = KMeans(n_clusters=k, random_state=0)
reporting_summary['cluster'] = kmeans.fit_predict(X)

# Analyze the resulting clusters
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
reporting_summary['cluster_center'] = cluster_centers[reporting_summary['cluster']]

# Print the cluster centers and the news sources in each cluster
print("Cluster Centers:")
print(pd.DataFrame(cluster_centers, columns=['event_count']))

print("\nNews Sources in Each Cluster:")
for cluster_label, group in reporting_summary.groupby('cluster'):
    print(f"Cluster {cluster_label}:")
    print(group[['source_name', 'event_count']])
    print()